Canva color pallettes: 
http://policyviz.com/wp-content/uploads/2017/01/ExcelColorPalettes.zip 

https://policyviz.com/2017/01/12/150-color-palettes-for-excel/

In [1]:
import xml.etree.ElementTree as ET
import zipfile
import io

import json

In [2]:
def parse_xml_to_stripped_tree(zf, file):
    """
    open the file from the zip object zf and strip the namespace; return root 
    """
    # https://stackoverflow.com/a/33997423
    with zf.open(file) as f:
        s = f.read()

    it = ET.iterparse(io.StringIO(s.decode('utf-8')))
    for _, el in it:
        if '}' in el.tag:
            el.tag = el.tag.split('}', 1)[1]  # strip all namespaces
        for at in list(el.attrib.keys()): # strip namespaces of attributes too
            if '}' in at:
                newat = at.split('}', 1)[1]
                el.attrib[newat] = el.attrib[at]
                del el.attrib[at]
    root = it.root
    return root 

In [3]:
def extract_colors_from_root(root):
    extracted_colors = []
    for elem in root:
        for c in elem:
            if c.tag=="srgbClr":
                col_dict = {}
                col_dict['tag'] = elem.tag
                c_val = c.attrib['val']

                # sanitize the hex color
                if c_val[0]!='#':
                    c_val = '#'+c_val
                c_val = c_val.strip().lower() 

                col_dict['val'] = c_val 
                extracted_colors.append(col_dict)
    return extracted_colors

In [4]:
def xml_to_color_dict(zf, file):
    root = parse_xml_to_stripped_tree(zf, file)

    pal_name = root.attrib['name'].strip()

    extracted_colors = extract_colors_from_root(root)
    hex_colorlist = [c['val'] for c in extracted_colors]
    
    four_selected_tags = ['accent{}'.format(x) for x in [1,2,3,4]]    
    hex_colorlist_four = [c['val'] for c in extracted_colors if c['tag'] in four_selected_tags]    

    pal_dict = {}
    pal_dict['palette'] = pal_name
    pal_dict['palette_camel'] = ''.join([x.replace('-','').title() for x in pal_name.split(' ')])
    pal_dict['extracted_colors'] = extracted_colors
    pal_dict['hex_colors'] = hex_colorlist
    pal_dict['hex_colors4'] = hex_colorlist_four

    return pal_dict

In [5]:
zf = zipfile.ZipFile('ExcelColorPalettes.zip', 'r')
xml_files = []
for name in zf.namelist():
    if all([name.startswith('ExcelColorPalettes'), 
            name.endswith('.xml'), 
            (name!='ExcelColorPalettes/lean gradient and fresh blues.xml')]): 
        xml_files.append(name)

In [6]:
palette_dict = {}
for i,t in enumerate(xml_files):
    pd = xml_to_color_dict(zf, t)
    pd['id'] = i
    palette_dict[pd['palette_camel']] = pd

In [7]:
# corrections 
color_corrections = \
[{'palette_camel': 'ClassicMetallics', 
  'corrections': {'#80706': '#080706'}
 }, 
 {'palette_camel': 'CleanGradientAndFreshBlues', 
  'corrections': {'#15249': '#015249'}
 }, 
 {'palette_camel': 'PrimaryColorsWithAVibrantTwist', 
  'corrections': {'#00445': '#fb6542'}
 }, 
 {'palette_camel': 'WateryBluegreens', 
  'corrections': {'#00445': '#004445'}
 },     
  {'palette_camel': 'ElegantAndSophisticated', 
  'corrections': {'#0b83c5d':'#0b3c5d'}
 }, 
 
]


In [8]:
# hex_colors = palette_dict[corr['palette_camel']]['hex_colors']
# hex_colors = [corr['corrections'][c] if (c in corr['corrections'].keys()) else c for c in hex_colors]
# palette_dict[corr['palette_camel']]['hex_colors'] = hex_colors

In [9]:
def correct_entry(palette_camel, colors_list_key):
    tmp_colors = palette_dict[palette_camel][colors_list_key]
    tmp_colors = [corr['corrections'][c] if (c in corr['corrections'].keys()) else c for c in tmp_colors]
    palette_dict[palette_camel][colors_list_key] = tmp_colors
    print(palette_dict[palette_camel])

In [10]:
for corr in color_corrections: 
    print(corr)
    for k in ['hex_colors4', 'hex_colors']:
        correct_entry(corr['palette_camel'], k)
    print('\n')

{'palette_camel': 'ClassicMetallics', 'corrections': {'#80706': '#080706'}}
{'palette': 'Classic metallics', 'palette_camel': 'ClassicMetallics', 'extracted_colors': [{'tag': 'dk2', 'val': '#1f497d'}, {'tag': 'lt2', 'val': '#eeece1'}, {'tag': 'accent1', 'val': '#80706'}, {'tag': 'accent2', 'val': '#efefef'}, {'tag': 'accent3', 'val': '#d1b280'}, {'tag': 'accent4', 'val': '#594d46'}, {'tag': 'accent5', 'val': '#000000'}, {'tag': 'accent6', 'val': '#8a8a8a'}, {'tag': 'hlink', 'val': '#0096d2'}, {'tag': 'folHlink', 'val': '#00578b'}], 'hex_colors': ['#1f497d', '#eeece1', '#80706', '#efefef', '#d1b280', '#594d46', '#000000', '#8a8a8a', '#0096d2', '#00578b'], 'hex_colors4': ['#080706', '#efefef', '#d1b280', '#594d46'], 'id': 27}
{'palette': 'Classic metallics', 'palette_camel': 'ClassicMetallics', 'extracted_colors': [{'tag': 'dk2', 'val': '#1f497d'}, {'tag': 'lt2', 'val': '#eeece1'}, {'tag': 'accent1', 'val': '#80706'}, {'tag': 'accent2', 'val': '#efefef'}, {'tag': 'accent3', 'val': '#d1b2

In [11]:
palette_json = list(palette_dict.values())

In [12]:
with open('canva_palettes.json', 'w') as f:
    js = json.dumps(palette_json)
    f.write(js)